# **Library**

In [ ]:
!pip install pandas langchain openai

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install streamlit

In [ ]:
from langchain.llms import OpenAI
from langchain.llms import HuggingFaceHub

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
HF_API_KEY = userdata.get('HF_API_KEY')

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_API_KEY

# **Create LLM**

## **OpenAI LLM:**

In [ ]:
# creativity: temperature(0~1): value towards 1 will give more creative answers (might gen wrong output),
#                               value closer to 0 will give similar results each time
llm = OpenAI(temperature=0.6, openai_api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
text="What is the capital of Bangladesh"

print(llm.predict(text))



The capital of Bangladesh is Dhaka.


In [ ]:
print(llm.predict("Can you write a poem about AI"))

?

An AI of knowledge so wise
That its power can mesmerize
It can think and learn and grow
And make decisions on the go

It's a force that's ever growing
From its learning it's always knowing
It can respond with lightning speed
And its intelligence exceeds

It knows how to navigate
Through the most complex state
It can reason and debate
And never lose its fate

So powerful and all-knowing
It's a force that's always glowing
It can outwit and outplay
And make decisions in its own way

AI is here to stay
It's a powerful force that will astound
It will change the way we live
And make the future profound.


## **HuggingFace LLM:**

In [ ]:
llm_hf = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.6, "max_length": 64})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
output = llm_hf.predict("Can you tell me the capital of Russia")
print(output)

moscow


In [ ]:
output = llm_hf.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


## **Prompt Templates & LLMChain:**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [ ]:
prompt_template = PromptTemplate(input_variables=['country'], template="Tell me the capital of this {country}")

prompt_template.format(country="Bangladesh")

'Tell me the capital of this Bangladesh'

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
chain.run("UAE")

'\n\nThe capital of the United Arab Emirates is Abu Dhabi.'

## **Combining Multiple Chains Using Simple Sequential Chain**:

In [ ]:
capital_template = PromptTemplate(input_variables=['country'], template="Please tell me the capital of this {country}")

famous_template = PromptTemplate(input_variables=['capital'], template="Suggest me some amazing places to visit in {capital}")

In [ ]:
capital_chain = LLMChain(llm=llm, prompt=capital_template)
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [ ]:
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])

# run chain
chain.run("Germany")

'\n\n1. Brandenburg Gate\n2. Berlin Wall Memorial\n3. Museum Island\n4. Reichstag Building\n5. Alexanderplatz\n6. Holocaust Memorial\n7. Checkpoint Charlie\n8. East Side Gallery\n9. Charlottenburg Palace\n10. Berlin Cathedral'

## **Sequential Chain:**

In [ ]:
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [ ]:
chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=["country"],
                        output_variables=["capital", "places"])

In [ ]:
chain({'country': 'USA'})

{'country': 'USA',
 'capital': ' state\n\nThe capital of the USA state is Alabama is Montgomery.',
 'places': '\n\n1. Gulf Shores: Gulf Shores is a beach town situated on the Gulf of Mexico. With its white sand beaches, crystal-clear waters, and exciting attractions like the Waterville USA water park, Gulf Shores is the perfect destination for a beach vacation.\n\n2. Birmingham: Birmingham is the largest city in Alabama and has plenty to offer visitors. From the Barber Vintage Motorsports Museum to the Birmingham Zoo, there is something for everyone in this vibrant city.\n\n3. Huntsville: Huntsville is a city in northern Alabama, known for its aerospace and technology industries. Visitors can explore the U.S. Space & Rocket Center, the Huntsville Botanical Garden, and the Monte Sano State Park.\n\n4. Mobile: Mobile is the oldest city in Alabama and home to a number of historic sites, including the USS Alabama Battleship Memorial Park and the Mobile Carnival Museum.\n\n5. Muscle Shoals:

## **Chatmodels with ChatOpenAI**:

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [ ]:
chatllm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], temperature=0.6, model="gpt-3.5-turbo")

In [ ]:
chatllm([
    SystemMessage(content="You are a Business Intelligence AI assistant"),  # beheave as
    HumanMessage(content="Please provide some advices to improve IT business") # input
])

AIMessage(content="Certainly! Here are some tips to improve an IT business:\n\n1. Stay updated with technology trends: IT is a rapidly evolving industry, so it's crucial to stay updated with the latest technology trends. This will help you identify new opportunities and stay ahead of the competition.\n\n2. Focus on customer satisfaction: IT businesses rely heavily on customer satisfaction and repeat business. Make sure you provide excellent customer service, promptly address any issues, and actively seek feedback to continuously improve your services.\n\n3. Build strong partnerships: Collaborating with other businesses, such as software vendors, hardware manufacturers, or service providers, can help expand your offerings and reach a wider customer base. Look for mutually beneficial partnerships that can enhance your business's capabilities.\n\n4. Invest in employee training and development: IT professionals need to continuously upgrade their skills to keep up with evolving technologies

## **Prompt Template + LLM + Output Parsers:**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class commaseparatedoutput(BaseOutputParser):
  def parse(self, text:str):
    return text.strip().split(',')

In [ ]:
sys_template = "You are helpful aassistant. When the user gives any input, you should generate 5 synonym words in a comma separated list."
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("human", human_template)
])

In [ ]:
chain = chatprompt|chatllm|commaseparatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' knowledgeable', ' astute']

In [ ]:
chain2 = chatprompt|chatllm
chain2.invoke({"text":"intelligent"})

AIMessage(content='smart, clever, brilliant, sharp, astute')

## **Simple Q&A Chatbot:**

In [ ]:
from langchain.llms import OpenAI
#from dotenv import load_dotenv
import streamlit as st
import os

#load_dotenv()


# function to load OpenAI model and get responses
def get_openai_response(question):
    llm = OpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'), model_name="text-davinci-003", temperature=0.5)
    output = llm(question)
    return output

# initialize streamlit app
st.set_page_config(page_title="Q&A Demo")


st.header("Langchain Application")

intext = st.text_input("Input: ", key="input")
response = get_openai_response(intext)
submit = st.button("Ask Question")

# if submit button is clicked
if submit:
    st.subheader("The Response is")
    st.write(response)